# Lecture 18

## Thursday, November 1st 2018

* Generators
* How things work

# Generators

## Example

In [ ]:
def mygen(N):
    for i in range(N):
        yield i**2

for vals in mygen(7):
    print(vals)

* A generator function looks like a normal function, but yields values instead of returning them.

* The syntax is the same otherwise ([PEP 255 -- Simple Generators](https://www.python.org/dev/peps/pep-0255/)).

* A generator is different; when the function runs, it creates a generator.

* The generator is an iterator and gets an internal implementation of `__iter__` and `__next__`.

* When `next` is called on the generator, the function proceeds until the first `yield`.

* The function body is now suspended and the value in the `yield` is then passed to the calling scope as the outcome of the `next`.

* When next is called again, it gets `__next__` called again (implicitly) in the generator, and the next value is yielded.

* This continues until we reach the end of the function, the return of which creates a `StopIteration` in next.

Any Python function that has the `yield` keyword in its body is a generator function.

In [ ]:
def gen123():
    print("A")
    yield 1
    print("B")
    yield 2
    print("C")
    yield 3

g = gen123()

print(gen123)
print(type(gen123))
print(type(g))

In [ ]:
print("A generator is an iterator.")
print("It has {} and {}".format(g.__iter__, g.__next__))

In [ ]:
print(next(g))

In [ ]:
print(next(g))

In [ ]:
print(next(g))

In [ ]:
print(next(g))

### More notes on generators
* Generators yield one item at a time
* In this way, they feed the `for` loop one item at a time

In [ ]:
for i in gen123():
    print(i, "\n")

## Exercise 1

#### Do the following exercise in a `.py` file called `sentence.py`.

Create a `Sentence` iterator class that uses a generator.

**Notes and Hints:** 
* You will write the generator in the `__iter__` special method.
* The generator automatically gets `__next__` (no need to implement it).
* Now you only need to create a single class.

```python
class SentenceIterator:
    def __init__(self, words): 
        self.words = words 
        self.index = 0
        
    def __next__(self): 
        try:
            word = self.words[self.index] 
        except IndexError:
            raise StopIteration() 
        self.index += 1
        return word 

    def __iter__(self):
        return self

class Sentence: # An iterable
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __iter__(self):
        return SentenceIterator(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
```

# How Things Work
We've introduced some data structures and discussed why they're important.

Now we'll go over where things live in the computer and how `Python` actually does things.

## Process address space

What do we mean when we say a program is "in memory"?

- compiled code must be loaded from disk into memory. 
- once your program starts, it must create (reserve) space for the variables it will use and it must store and read values from those variables. 

The code, the reserved space, and the generated data all constitute a program's memory footprint.



Every operating system has a (different) convention for exactly where and how these different resources are actually laid out in memory. These conventions are called *object file formats*.

In Linux, the most common object format is called `ELF`, short for "Executable and Linkable Format".

A simplified view of our example program looks like this in memory: the `stack` and the `heap`
  - [Stack and Heap](https://stackoverflow.com/questions/79923/what-and-where-are-the-stack-and-heap).

### The Stack

The stack keeps track of function calls and their data. 

- Whenever a function is called, a new chunk of memory is reserved at the top of the stack in order to store variables for that function. 

- This includes variables that you have defined inside your function and function parameters, but it also includes data that were generated automatically by the compiler. 

- The most recognizable value in this latter category is the return address. When a function calls `return`, the computer starts executing instructions at the location that the function was originally called from.

- When a function returns, it removes its stack frame from the stack. This means that at any given point, the stack contains a record of which functions the program is currently in.

- Removing the function stack from the stack becomes a problem if you want to create space for variables and then access them after the function returns. 

### The Heap
- The heap is another memory location which is not reclaimed after a function returns. It is explicitly managed, which means that you need to ask to allocate a variable in it. What's more, when you're finished with that space, you need to say so.

- This interface, explicitly requesting and releasing memory from the heap, is typically called *memory management*.

- In `C` you do this directly using `malloc`/`free`. Python takes care of this for you using **garbage collection**.

## How does Python manage memory?
So far, when you create a list in `Python` (e.g. `a=[1,2,3,4,5]`), we have been thinking of it as an array of integers.

But that is not actually the case.

An array would be 5 contiguous integers with some book-keeping at the beginning either on the stack or the heap.

You are all highly encouraged to read the eminently readable article [Why Python is Slow](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/) from Jake Vanderplas's blog.  Most of the following points come from his discussion.

### `Python` objects are allocated on the heap: an `int` is not an `int`
An `int` is represented in `Python` as a `C` structure **allocated on the heap**.

The picture of this `C` structure looks a little bit like this:

![](http://jakevdp.github.io/images/cint_vs_pyint.png)

On Jake's blog, he shows how this integer is represented:

```C
struct _longobject {
    long ob_refcnt; // in PyObject_HEAD
    PyTypeObject *ob_type; // in PyObject_HEAD
    size_t ob_size; // in PyObject_HEAD
    long ob_digit[1];
};
```
It's not just a simple integer!

### Boxing and unboxing (or why `Python` is slow)
Because `int`s are stored in this scheme, a simple addition involves a lot of work!

    
So it's not a simple addition...there is all this machinery around it.

And that example was just to add two `int`s where `binary_add` is optimized in `C`!

If these were user-defined classes, there would be additional overhead from `dunder` methods for addition!

### What About Lists?

A `Python` list is represented by the following `struct`:

```C
typedef struct {
    long ob_refcnt;
    PyTypeObject *ob_type;
    Py_ssize_t ob_size;
    PyObject **ob_item;
    long allocated;
} PyListObject;
```

Notice the `PyObject**`. This points to the contents of the list. What is this a list of? This is a list of `PyObject*`.

Each of those pointers, when dereferenced, gives us an `IntStruct`. The `ob_size` tells us the number of items on the list.

Thus this is **an array of pointers to heap allocated `IntStruct`**s: [Simple Example](https://goo.gl/jjfpNV).

## What do others do?

`Julia`:
>In the most general case, an array may contain objects of type Any. For most computational purposes, arrays should contain objects of a more specific type, such as Float64 or Int32.

```julia
a = Real[]    # typeof(a) = Array{Real,1}
if (f = rand()) < .8
    push!(a, f) # f will always be Float64
end
```

Because `a` is a an array of abstract type `Real`, it must be able to hold any `Real` value. Since `Real` objects can be of arbitrary size and structure, `a` must be represented as an array of pointers to individually allocated `Real` objects. We should instead use:

```julia
a = Float64[] # typeof(a) = Array{Float64,1}
```

which will create a contiguous block of 64-bit floating-point values.

`C`:

You allocate explicitly either on the heap or stack.

In `Python` you can append to lists. So what's an `ob_size` doing in our struct then?

```C
typedef struct {
    long ob_refcnt;
    PyTypeObject *ob_type;
    Py_ssize_t ob_size;
    PyObject **ob_item;
    long allocated;
} PyListObject;
```

Turns out Python lists are implemented in something called a dynamic array.

## Arrays

A static array is a contiguous slab of memory of known size, such that `n` items can fit in.  This is a great data structure. Why?

- constant time index access: `a[i]` is $\mathcal{O}(1)$
  - Just seek `i * sizeof(int)`

- linear time traversal or search: $1$ unit / loop iteration means $\mathcal{O}(n)$ in loop.

- locality in memory: it's one `int` after another

Tuples in `Python` are fixed size, static arrays.

But the big problem is, what if we want to add something more beyond the end of the array?

Then we must use dynamic arrays.

## Dynamic Arrays
What `Python` does is first create a fixed size array of these `Pyobject*` pointers on the heap. Then, as you append, it uses its own algorithm to figure out when to expand the size of the array.

[`listobject.c`](https://svn.python.org/projects/python/trunk/Objects/listobject.c )

```C
/* This over-allocates proportional to the list size, making
 * room for additional growth.  The over-allocation is mild, 
 * but is enough to give linear-time amortized behavior over a 
 * long sequence of appends() in the presence of a poorly- 
 * performing system realloc().  The growth pattern is:  0, 4, 
 * 8, 16, 25, 35, 46, 58, 72, 88, ...
*/
new_allocated = (newsize >> 3) + (newsize < 9 ? 3 : 6);
```

### Performance of Dynamic Arrays

Lets assume we start with an array of size of $1$ (one slot) and then double the size each time. After $n$ doublings, we have an array with $2^n$ slots.  So, it then takes $lg(n)$ doublings for the array to have $n$ slots (note, $\lg(n)$ means $\log_{2}(n)$).

Notice that we might not get the continuously allocated memory we want. So we'll have to recopy to a larger array.

The last $n/2$ numbers in the array don't move at all (they're the new ones). The previous $n/4$ numbers in the array would have moved once, the previous $n/8$ twice, and so on.

Therefore, the $i$-th chunk of numbers will have moved $$i \dfrac{n}{2^{i+1}}$$ times.

We need to add all these movements up.

Thus the total number of movements is 

$$\sum_{i=1}^{lg(n)} i\frac{n}{2^{i+1}} \leq \frac{n}{2} \sum_{i=1}^{\infty} \frac{i}{2^i} = n.$$


This is an amazing result. The work of reallocation is still $\mathcal{O}(n)$ on the average, as if a single array had been allocated in advance!

Here's the calculation.

\begin{align*}
  \sum_{i=0}^{\infty}{\left(i+1\right)x^{i+1}} &= x\sum_{i=0}^{\infty}{\left(i+1\right)x^{i}} \\
      &= x\frac{\mathrm{d}}{\mathrm{d}x}\sum_{i=0}^{\infty}{x^{i+1}} \\
      &= x\frac{\mathrm{d}}{\mathrm{d}x}\left[x\sum_{i=0}^{\infty}{x^{i}}\right] \\
      &= x\frac{\mathrm{d}}{\mathrm{d}x}\left[\frac{x}{1-x}\right] = \frac{x}{\left(1-x\right)^{2}}.
\end{align*}

When $x = 1/2$ we have
$$\sum_{i=0}^{\infty}{\left(i+1\right)x^{i+1}} = 2.$$
From here we can easily get the result from the previous slide.

## Containers  vs Flats

Earlier we saw how `Python` lists contained references to integer ("digit")+metdata based structs on the heap. 

We call sequences that hold such "references" to objects on the heap **Container Sequences**. Examples of such container sequences are `list`, `tuple`, `collections.deque`.

There are collections in `Python` which contain contiguous "typed" memory (which itself is allocated on the heap). We call these **Flat Sequences**. Such containers in `Python 3` are: `str`, `bytes`, `bytearray`, `memoryview`, `array.array`.

You have probably extensively used a type of flat sequence not mentioned yet. This is `NumPy`'s ndarray: `np.array`.

All of these are faster as they work with **contiguous blocks of uniformly formatted memory**.

From Fluent Python:
> Container sequences hold references to the objects they contain, which may be of any type, while flat sequences physically store the value of each item within its own memory space, and not as distinct objects. Thus, flat sequences are more compact, but they are limited to holding primitive values like characters, bytes, and numbers.

This is also a more general way of thinking about data structures. 

>**Contiguously-allocated** structures are composed of single slabs of memory, and include arrays, matrices, heaps, and hash tables.

>**Linked** data structures are composed of distinct chunks of memory bound together by pointers, and include lists, trees, and graph adjacency lists.

(Steven S Skiena. The Algorithm Design Manual)

- A critical advantage of something like a contiguous memory array is that indexing is a constant time operation, as opposed to worst-case $\mathcal{O}(n)$, as we saw in linked lists. 
- Other benefits include a tighter size and a locality of memory which benefits cache and general memory transport.

## Mutable vs Immutable

The mutability of objects has recurred often in this course.

One can also study containers based on their mutability. 

**Mutable sequences** in `Python 3` are: `list, bytearray, array.array, collections.deque, memoryview`.

**Immutable sequences** in `Python 3` are `tuple, str, bytes`.

Lets learn about some of these collections in `Python`.

### array.array
The `list` type is nice and very flexible, but if you need to store many many (millions) of floating point variables, `array.array` is a better option.

`array.array` stores just the bytes representing the type, so its just like a contiguous `C` array of things in RAM, and also just like a `numpy` array. 

`array.array` *is* mutable, and you don't need to allocate ahead of time (reallocation will be done).

In [ ]:
from array import array
from random import random
#generator expression instead of list comprehension
floats_aa = array('d', (random() for i in range(10**8)))
print(floats_aa.itemsize, "   ", type(floats_aa), "   ", floats_aa[5])

Now let's do the same thing with a list comprehension:

In [ ]:
floats_list = [random() for i in range(10**8)]

In [ ]:
%%time
for f in floats_aa:
    pass

In [ ]:
%%time
for f in floats_list:
    pass

### A Few Observations
* Looks like a regular `Python` list on 100 million floats only costs double.
* Why is accessing floats in an `array.array` so slow?
  - Because each float is *boxed* by the `Python` runtime. You saw this earlier!

In an `array.array` or in `Numpy` for that matter, when you "iterate" over the array, and use the `int`s you get, what `Python` does is to take that 32 bits or 64 bits from memory, wrap it up into one of these structs, and hand it to you. You asked for a `Python` `int` after all.

Operations on `array.array` which can be done with `C` are fast; access into `Python` is slow. 

This is why `numpy.ndarray` is written in `C`, with operations like `numpy.dot` written in `C`. 

(None of the `array.array` functionality is exposed with any complex operations under the hood, so its current use remains limited.)

If you want to use numerical stuff, use `numpy` arrays. 

See https://www.python.org/doc/essays/list2str/ for a discussion on when it's a good idea to use `array.array`.

### memoryviews
Memoryviews, inspired by `NumPy` and `SciPy`, let you handle slices of arrays without expensively copying bytes.

Travis Oliphant, as quoted in Fluent:
>A memoryview is essentially a generalized NumPy array structure in Python itself (without the math). It allows you to share memory between data-structures (things like PIL images, SQLlite databases, NumPy arrays, etc.) without first copying. This is very important for large data sets.